# Box Model of the Salish Sea
 - Schematic of box model can be found in Supplementary Info Figure 3.1
---------------------

In [1]:
import sys
import os
import importlib
import xarray as xr
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from scipy import optimize
import math

# Open model simulations and observational data files

In [2]:
sys.path.insert(1, '../Functions')
import Output_Cleanup_Functions as open_files
importlib.reload(open_files)

<module 'Output_Cleanup_Functions' from '/raid/mmstoll/Code/Industrial_Era_OA_Code/../Functions/Output_Cleanup_Functions.py'>

In [3]:
file_historic_12km = '/home/mmstoll/raid/Data/CCModel_Historic_Data/usw12_avg_ctracers_vintp_ROMS.nc'
file_modern_12km = '/home/mmstoll/raid/Data/CCModel_Modern_Data/usw12_avg_ctracers_vintp.nc'

In [4]:
ds_modern, ds_historic, month_list_modern, month_list_historic = open_files.open_files_12vs12(file_modern_12km, file_historic_12km)
depth_list = open_files.depths()

Specify start and end dates for modern and historic model simulations

In [5]:
date_index_modern_start = month_list_modern.index('December 2000')
date_index_modern_end = month_list_modern.index('November 2007')
date_index_historic_start = month_list_historic.index('December 1897')
date_index_historic_end = month_list_historic.index('November 1904')

Observational bottle data from the Salish Sea collected by the Washington Ocean Acidification Center

In [6]:
file_SalishSea_obs_data = '/home/mmstoll/raid/Data/Coral_Data/WOAC_DIC_Alk_pH_data_Mary_Margaret_Stoll_2008-2018.xlsx'
SS_obs = pd.read_excel(file_SalishSea_obs_data, engine = 'openpyxl')
SS_obs = SS_obs[(SS_obs['DIC_UMOL_KG'].notna()) & (SS_obs['pHTOTAL'].notna())]

# Determine end member properties for box model of the Salish Sea 

### Functions to calculate carbonate chemistry parameters

#### Function to calculate $p$CO$_2$ from DIC and Alk

In [7]:
def calc_pco2_TA_DIC(temp, sal, DIC, Alk):
    TK = temp + 273.16 #temperature in Celsius into Kelvin
    I = 19.924*sal/(1000 - 1.005 * sal) #molal ionic strength
    R = 83.131 #Gas constant
    
    #Conservative seawater components as a function of salinity
    B_T = 0.000416 * sal / 35 #total boron
    F_T = 7e-5 * sal / 35 #total HF and F-
    # SO4_T = .0293*Sal/35 #total sulfate species from Millero 1995
    SO4_T = .02824 * sal / 35 #total sulfate species from DOE 1994, and Dickson/Millero's refitting of Mehrbach
    Ca_T = .01028 * sal / 35

    #1st dissociation constant for seawater as a function of T and S
    #Seawater Scale
    #Fortran code - Millero p.664 (1995) using Mehrbach et al. data on seawater scale
    K_1 = 10**(-1*(3670.7*(1/TK) - 62.008 + 9.7944*math.log(TK) - 0.0118*sal + 0.000116*sal**2))

    #2nd dissociation constant for seawater as a function of T and S
    #Seawater Scale
    #Fortran code - Millero p.664 (1995) using Mehrbach et al. data on seawater scale
    K_2 = 10**(-1*(1394.7*(1/TK) + 4.777 - 0.0184*sal + 0.000118*sal**2))

    #dissociation contant for H2O
    #Seawater Scale
    #Fortran code - Millero p.670 (1995) using composite data
    K_W = math.exp(-13847.26*(1/TK) + 148.9652 - 23.6521 * math.log(TK) + (118.67*(1/TK) - 5.977 + 1.0495 * math.log(TK)) * math.sqrt(sal) - 0.01615 * sal)
               
    #Boric Acid
    #Fortran code - Millero p.669 (1995) using data from Dickson (1990)
    K_B = math.exp((-8966.90 - 2890.53*math.sqrt(sal) - 77.942*sal + 1.728*(sal**1.5)-0.0996*sal**2)*(1/TK) + (148.0248 + 137.1942*math.sqrt(sal) +1.62142*sal) + (-24.4344 -25.085*math.sqrt(sal) - 0.2474*sal)*math.log(TK)+0.053105*math.sqrt(sal)*TK)

    #CO2 solubility in water
    #Weiss 1974
    K_0 = math.exp(93.4517/(TK/100) - 60.2409 + 23.3585*math.log(TK/100) + sal
                    *(.023517 - 0.023656 * (TK/100) + 0.0047036 * (TK/100)**2))
    
    def f_zero(x):
        denom = (x**2+K_1*x+K_1*K_2)
        Alpha0 = (x**2)/denom
        Alpha1 = (K_1*x)/denom
        Alpha2 = (K_1*K_2)/denom
    
        return((Alpha1*DIC) + (2*Alpha2*DIC) + (K_W/x) + (B_T/(1+x/K_B)) - Alk - x) #equation that equals 0

    root = optimize.bisect(f_zero,1*10**-16,1) #calculates root of the equation returned by f_zero
    ph_value = (-math.log(root,10)) #calculates and returns the pH values

    denom = (root**2 + K_1*root + K_1*K_2)
    bicarbonate = DIC*10**6*K_1*root / denom
    carbonate = DIC*10**6*K_1*K_2 / denom
    CO2_star = (root*bicarbonate)/K_1
    pCO2 = CO2_star / K_0

    return(pCO2)

#### Function to calculate DIC from Alk and $p$CO$_2$

In [8]:
def calc_DIC(temp, sal, Alk, pCO2):
    TK = temp + 273.16 #temperature in Celsius into Kelvin
    I = 19.924*sal/(1000 - 1.005 * sal) #molal ionic strength
    R = 83.131 #Gas constant
    
    #Conservative seawater components as a function of salinity
    B_T = 0.000416 * sal / 35 #total boron
    F_T = 7e-5 * sal / 35 #total HF and F-
    # SO4_T = .0293*Sal/35 #total sulfate species from Millero 1995
    SO4_T = .02824 * sal / 35 #total sulfate species from DOE 1994, and Dickson/Millero's refitting of Mehrbach
    Ca_T = .01028 * sal / 35

    #1st dissociation constant for seawater as a function of T and S
    #Seawater Scale
    #Fortran code - Millero p.664 (1995) using Mehrbach et al. data on seawater scale
    K_1 = 10**(-1*(3670.7*(1/TK) - 62.008 + 9.7944*math.log(TK) - 0.0118*sal + 0.000116*sal**2))

    #2nd dissociation constant for seawater as a function of T and S
    #Seawater Scale
    #Fortran code - Millero p.664 (1995) using Mehrbach et al. data on seawater scale
    K_2 = 10**(-1*(1394.7*(1/TK) + 4.777 - 0.0184*sal + 0.000118*sal**2))

    #dissociation contant for H2O
    #Seawater Scale, CONSISTENT
    #Fortran code - Millero p.670 (1995) using composite data
    K_W = math.exp(-13847.26*(1/TK) + 148.9652 - 23.6521 * math.log(TK) + (118.67*(1/TK) - 5.977 + 1.0495 * math.log(TK)) * math.sqrt(sal) - 0.01615 * sal)
               
    #Boric Acid
    #Fortran code - Millero p.669 (1995) using data from Dickson (1990)
    K_B = math.exp((-8966.90 - 2890.53*math.sqrt(sal) - 77.942*sal + 1.728*(sal**1.5)-0.0996*sal**2)*(1/TK) + (148.0248 + 137.1942*math.sqrt(sal) +1.62142*sal) + (-24.4344 -25.085*math.sqrt(sal) - 0.2474*sal)*math.log(TK)+0.053105*math.sqrt(sal)*TK)

    #CO2 solubility in water
    #Weiss 1974
    K_0 = math.exp(93.4517/(TK/100) - 60.2409 + 23.3585*math.log(TK/100) + sal
                    *(.023517 - 0.023656 * (TK/100) + 0.0047036 * (TK/100)**2))
    
    def f_zero(x):
        lhs = Alk * x**2 * (K_B+x)
        rhs = (pCO2*K_0/(1*10**6))*(K_B+x)*(K_1*x + 2*K_1*K_2) + x**2*K_B*B_T + (K_B + x)*(K_W*x - x**3)
        return(lhs-rhs)
    
    root = optimize.bisect(f_zero,1*10**-16,1)
    ph_value = (-math.log(root,10)) #calculates and returns the pH values
    
    DIC = (pCO2*K_0/(1*10**6))*(1 + (K_1/root) + ((K_1*K_2)/root**2))
 
    return(DIC)

### Upwelled water mass properties

#### Determine summertime average properties of upwelled waters that enter the Strait of Juan de Fuca from the California Current in the historic and modern eras

In [9]:
#Model domain lat and lon indices at entrance of the Strait of Juan de Fuca
iy_mod = 317
ix_mod = 282

In [10]:
def is_jja(month, month_start, month_end):
        return (month >= month_start) & (month <= month_end)

In [11]:
month_start = 6
month_end = 8

mod_TA = ds_modern['Alk'][date_index_modern_start:date_index_modern_end,6:,iy_mod-2:iy_mod+3,
                          ix_mod-2:ix_mod+3]
mod_TA = mod_TA.sel(time=is_jja(mod_TA['time.month'], month_start, month_end)).mean()
    
mod_DIC = ds_modern['DIC'][date_index_modern_start:date_index_modern_end,6:,iy_mod-2:iy_mod+3,
                                     ix_mod-2:ix_mod+3]
mod_DIC = mod_DIC.sel(time=is_jja(mod_DIC['time.month'], month_start, month_end)).mean()
    
mod_temp = ds_modern['temp'][date_index_modern_start:date_index_modern_end,6:,iy_mod-2:iy_mod+3,
                                     ix_mod-2:ix_mod+3]
mod_temp = mod_temp.sel(time=is_jja(mod_temp['time.month'], month_start, month_end)).mean()
    
mod_salt = ds_modern['salt'][date_index_modern_start:date_index_modern_end,6:,iy_mod-2:iy_mod+3,
                             ix_mod-2:ix_mod+3]
mod_salt = mod_salt.sel(time=is_jja(mod_salt['time.month'], month_start, month_end)).mean()

mod_density = ds_modern['rho'][date_index_modern_start:date_index_modern_end,6:,iy_mod-2:iy_mod+3,
                               ix_mod-2:ix_mod+3]
mod_density = (mod_density.sel(time=is_jja(mod_density['time.month'], month_start, month_end))+1027.4).mean()

    
hist_TA = ds_historic['Alk'][date_index_modern_start:date_index_modern_end,6:,iy_mod-2:iy_mod+3,
                             ix_mod-2:ix_mod+3]
hist_TA = hist_TA.sel(time=is_jja(hist_TA['time.month'], month_start, month_end)).mean()
    
hist_DIC = ds_historic['DIC'][date_index_modern_start:date_index_modern_end,6:,iy_mod-2:iy_mod+3,
                              ix_mod-2:ix_mod+3]
hist_DIC = hist_DIC.sel(time=is_jja(hist_DIC['time.month'], month_start, month_end)).mean()
    
hist_temp = ds_historic['temp'][date_index_modern_start:date_index_modern_end,6:,iy_mod-2:iy_mod+3,
                                ix_mod-2:ix_mod+3]
hist_temp = hist_temp.sel(time=is_jja(hist_temp['time.month'], month_start, month_end)).mean()
    
hist_salt = ds_historic['salt'][date_index_modern_start:date_index_modern_end,6:,iy_mod-2:iy_mod+3,
                                ix_mod-2:ix_mod+3]
hist_salt = hist_salt.sel(time=is_jja(hist_salt['time.month'], month_start, month_end)).mean()
    
hist_density = ds_historic['rho'][date_index_modern_start:date_index_modern_end,6:,iy_mod-2:iy_mod+3,
                                  ix_mod-2:ix_mod+3]
hist_density = (hist_density.sel(time=is_jja(hist_density['time.month'], month_start, month_end))+1027.4).mean()    

#### Calculate DIC value of the upwelled waters that enter the Salish Sea in 2020
To account for the modern temporal offset (corals collected 2020; model output ~2005), during which rapid carbon accumulation occurred, we advance the modeled DIC of the California Current upwelled waters from 2005 to 2020 to match the collection date of the modern corals and assume that this advance follows the known rise in atmospheric CO$_2$. Given that the water entering the Salish Sea from the California Current is 25 years old (Murray et al. 2015), the water mass represented in the modern model simulation was last in contact with the atmosphere in 1980. To advance modeled DIC from 2005 to 2020, we assume the waters entering the Salish Sea in 2020 were last in contact with the atmosphere in 1995 (CO$_2$ atm 1995 = 360 µatm; ΔCO$_2$ atm 1980–1995 = 21 µatm). We quantify the impact of this anthropogenic carbon on DIC as described in Methods, Equations 3 and 4. 

In [12]:
atm_CO2_1900 = 295 #atmospheric CO2
atm_CO2_1995 = 360 #atmospheric CO2 1995 (the upwelled waters the enter the Salish Sea in 2020 have a water mass age = 25 yrs, so atmospheric contact age = 1995)
atm_CO2_2020 = 415 #atmospheric CO2 2020

calc_pref_hist_DIC = calc_DIC(hist_temp, hist_salt, (hist_TA)/(hist_density*1000), atm_CO2_1900) #mol/kg
calc_pref_mod_DIC = calc_DIC(mod_temp, mod_salt, (mod_TA)/(mod_density*1000), atm_CO2_1995) #mol/kg

In [13]:
pref_DIC_diff = (calc_pref_mod_DIC*mod_density*1000)-(calc_pref_hist_DIC*hist_density*1000)
# pref_DIC_diff #mmol/m3

In [14]:
#DIC of summertime upwelled waters entering the Salish Sea in 2020 with a water mass age of 25 years 
upwelled_DIC_2020 = hist_DIC + pref_DIC_diff #mmol/m3

### Salish Sea water mass properties
 - Washington Ocean Acidification Center station 21 is closest to Admiralty Inlet, where a majority of Salish Sea corals were located

In [15]:
stn_21_obs = SS_obs.loc[SS_obs['STN'] == 21]
stn_21_obs = stn_21_obs.loc[(SS_obs['MONTH'] == 7) & (SS_obs['PRESS (dbar)'] >50)] #Select July data at a depth > 50dbar, where corals reside

temp_SS_obs = stn_21_obs['TEMP (C, ITS-90)'].mean() #units C
sal_SS_obs = stn_21_obs['SAL (PSS-78)'].mean()
TA_SS_obs = stn_21_obs['TA_UMOL_KG'].mean()/(1*10**6) #units mol/kg

# Perform box model calculations to estimate $p$CO$_2$ of the Salish Sea in the historic and modern eras
 - Inputs:
   - DIC upwelled (historic and modern)
   - CO$_2$ atmosphere (historic and modern)
   - Total alkalinity (constant)
   - Temperature (constant)
   - Salinity (constant) <br>
<br>
   - Variables that can change:
     - Tau (residence time in days)

<br>
  - Zero finding method calculates DIC of the Salish Sea, which can then be used to calculate $p$CO$_2$
<br>
  - This calculation is performed in both the historic and modern eras to determine a box model estimate of $\Delta$$p$CO$_2$. This value is compared to the coral-based estimate of $\Delta$$p$CO$_2$ in the Salish Sea.


#### Calculate residence time of Strait of Juan de Fuca

In [16]:
volume = 150000*25000*150 #units m3, calculated using Google Earth
flow_rate = (0.66*(140) + 0.33*(20))*10**3 #units m3/s; (MacCready et al. 2021 and Sutherland et al. 2011)

per_time = (flow_rate/volume)*60*60*24 #units per day
tau = 1/per_time #units days

#### Calculate estimates of pH and $p$CO$_2$ using box model equation
 - Change variable 'era' to reflect historic or modern time period
 - Change variable 'tau' to reflect different residence times; 65 days

In [17]:
def box_model_solution(era, tau):
    density = 1024
    piston_velocity = 7 #m/day
    mixed_layer = 100 #meters

    #2020
    if era =='modern':
        temp = temp_SS_obs
        TK = temp + 273.15
        sal = sal_SS_obs
        Alk_molkg = TA_SS_obs #mol/kg
        K_0 = math.exp(93.4517/(TK/100) - 60.2409 + 23.3585*math.log(TK/100) + sal
                       *(.023517 - 0.023656 * (TK/100) + 0.0047036 * (TK/100)**2)) #CO2 solubility in water; Weiss 1974
        co2_atm_molkg = (atm_CO2_2020/(1*10**6))*K_0 #mol/kg
        DIC_upwelled_molkg = upwelled_DIC_2020/(density*1000) #mol/kg; model value advanced to 2020 using preformed test to account for water mass age

    #1900
    if era == 'historic':
        temp = temp_SS_obs
        TK = temp + 273.15
        sal = sal_SS_obs
        Alk_molkg = TA_SS_obs #mol/kg
        K_0 = math.exp(93.4517/(TK/100) - 60.2409 + 23.3585*math.log(TK/100) + sal
                       *(.023517 - 0.023656 * (TK/100) + 0.0047036 * (TK/100)**2)) #CO2 solubility in water; Weiss 1974
        co2_atm_molkg = (atm_CO2_1900/(1*10**6))*K_0 #mol/kg
        DIC_upwelled_molkg = hist_DIC/(density*1000) #mol/kg; model value

    output_array = []
    pco2_array = []

    DIC = np.linspace(2000,2200,400) #umol/kg
    for i in range(0,len(DIC)):
        DIC_guess = DIC[i]/(10**6) #mol/kg
        pco2_guess = calc_pco2_TA_DIC(temp, sal, DIC_guess, Alk_molkg)
        pco2_molkg = (pco2_guess*K_0)/10**6
        output = ((1/tau) * DIC_upwelled_molkg -(1/tau) * DIC_guess + (piston_velocity/mixed_layer) * (co2_atm_molkg-pco2_molkg))
        output_array.append(output)
        pco2_array.append(pco2_guess)
        
    zero_line = np.zeros_like(DIC)
    index = int(np.argwhere(np.diff(np.sign(zero_line - output_array).flatten())))
    
    DIC_value = DIC[index] #umol/kg
    pco2_value = pco2_array[index] #uatm
    
    return(DIC_value, pco2_value)

In [18]:
#Input 'era' as 'historic' or 'modern'
#Input value for tau (65 days)
hist_DIC_value, hist_pco2_value = box_model_solution('historic', 65)
mod_DIC_value, mod_pco2_value = box_model_solution('modern', 65)

In [19]:
print('Box model prediction of centennial change in pCO2 in Salish Sea = '+str(round(mod_pco2_value - hist_pco2_value)) +' uatm.')

Box model prediction of centennial change in pCO2 in Salish Sea = 185 uatm.
